In [1]:
%reset -f
import numpy as np
#import random as rm
import timeit
import itertools
%load_ext line_profiler
from SequenceSim import experiments_markov_binary

In [2]:
# does it work for a second order?
# simulate some probabilities

test_n = 50
test_order = 3


nucleotides = 'ATGC'

file = "data/MA0024.1.pfm"

# List with values that are transitioned to 
test_transition_states = ["A", "T", "G", "C"]

order_nucleotides = [''.join(p) for p in itertools.product(nucleotides, repeat = test_order)]

prob = np.repeat(1/len(order_nucleotides), len(order_nucleotides))
test_init_freq = dict(zip(order_nucleotides, prob))

#prob = [list(np.repeat(1/len(nucleotides), len(nucleotides))) for i in range(len(order_nucleotides))]
#test_transition_dic = dict(zip(order_nucleotides, prob))


prob = [list(np.random.uniform(low=1, high=2, size= len(test_transition_states))) for i in range(len(order_nucleotides))]
prob = [i/sum(i) for i in prob]
cases_transition_dic = dict(zip(order_nucleotides, prob))


prob = [list(np.random.uniform(low=1, high=2, size= len(test_transition_states))) for i in range(len(order_nucleotides))]
prob = [i/sum(i) for i in prob]
controls_transition_dic = dict(zip(order_nucleotides, prob))


cases_seq_parameters = {'init_freq': test_init_freq,\
                       'transition_dic': cases_transition_dic,\
                       'transition_states': test_transition_states,\
                       'n': test_n,\
                       'order': test_order}

controls_seq_parameters = {'init_freq': test_init_freq,\
                       'transition_dic': controls_transition_dic,\
                       'transition_states': test_transition_states,\
                       'n': test_n,\
                       'order': test_order}


In [11]:
#timeit.timeit('markov_simulations(test_seq_parameters, 10000)', globals = globals(), number = 1)
#size = 100
#X = markov_simulations(test_seq_parameters, size)
#y = np.random.choice(a = [1, 0], p = [0.5, 0.5], size = size).reshape(size, 1)

X, y = experiments_markov_binary(cases_seq_parameters, controls_seq_parameters, 500, 500, cases_motif_file = file, cases_index = [25, 30])

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

#print(np.array(X))

In [12]:
#import keras
#from keras.models import Sequential
#from keras.layers import Dense, Flatten
#from keras.layers import Conv1D, MaxPooling1D

import keras.layers as kl
from keras.models import Sequential

In [13]:

# construct the model ----------------------------------------------------------
kl.model = Sequential()
kl.model.add(kl.Conv1D(20, kernel_size=(4), activation='relu', input_shape=(test_n, 4), padding='same'))
#kl.model.add(kl.Dense(100, input_shape=(test_n, 4), activation='relu'))
kl.model.add(kl.MaxPooling1D(pool_size=5))
kl.model.add(kl.Flatten())
#kl.model.add(kl.Dense(50, activation = 'relu'))
#kl.model.add(kl.Dense(10, activation = 'relu'))
kl.model.add(kl.Dense(1, activation='sigmoid'))
# Compile model
kl.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# compile
kl.model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# print model summary
kl.model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 50, 20)            340       
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 10, 20)            0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 541 (2.11 KB)
Trainable params: 541 (2.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:

# Train ------------------------------------------------------------------------
kl.model.fit(X_train, y_train,
          verbose=1,
             epochs = 10, 
          validation_data=(X_test, y_test))

Epoch 1/10
22/22 [==============================] - 1s 7ms/step - loss: 0.7089 - accuracy: 0.5343 - val_loss: 0.6784 - val_accuracy: 0.5700
Epoch 2/10
22/22 [==============================] - 0s 3ms/step - loss: 0.6637 - accuracy: 0.6100 - val_loss: 0.6476 - val_accuracy: 0.7033
Epoch 3/10
22/22 [==============================] - 0s 3ms/step - loss: 0.6280 - accuracy: 0.7414 - val_loss: 0.6149 - val_accuracy: 0.7833
Epoch 4/10
22/22 [==============================] - 0s 3ms/step - loss: 0.5925 - accuracy: 0.8214 - val_loss: 0.5817 - val_accuracy: 0.8233
Epoch 5/10
22/22 [==============================] - 0s 3ms/step - loss: 0.5551 - accuracy: 0.8814 - val_loss: 0.5452 - val_accuracy: 0.8800
Epoch 6/10
22/22 [==============================] - 0s 3ms/step - loss: 0.5172 - accuracy: 0.9100 - val_loss: 0.5072 - val_accuracy: 0.8967
Epoch 7/10
22/22 [==============================] - 0s 3ms/step - loss: 0.4765 - accuracy: 0.9314 - val_loss: 0.4681 - val_accuracy: 0.9133
Epoch 8/10
22/22 [==